In [20]:
import os
os.getpid()

27136

In [21]:
import random
list2 = []
file_data = open('amazon_review_less_than_300_unbalanced.csv')
for row in file_data:
    list2.append(row)## Data pre-processing module
    
list3 = []
for i in range(len(list2)):
    list3.append((list2[i][-2],list2[i][:-3]))
    
random.seed(10)
random.shuffle(list3)

lenth = len(list3)
train_list = list3[0:int(lenth*0.8)]
test_list = list3[int(lenth*0.8):]




In [22]:

import torch
from torch.utils.data.sampler import BatchSampler, Sampler, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertModel, AdamW, BertConfig
from tqdm import tqdm

class Corpora():
    """
        A class that holds the texts and class labels.
    """
    
    # Modification: l represent train_list or test_list
    
    def __init__(self, l):
        """
            file_path: on MAGIC, it is either
                        data-badassnlp/cola_public/raw/in_domain_train.tsv or
                        data-badassnlp/cola_public/raw/in_domain_dev.tsv
                    on Google Colab, change them to the corresponding paths.
        """

        self.sentences = [s[1] for s in tqdm(l)]
        self.labels = [int(s[0]) for s in tqdm(l)]
        
        
class TextClassificationDataSet(Dataset):
    """
        Define a dataset consisting of pairs of (sequence_of_word_indices, class_label).
        class_label is either 0 or 1.
    """
    def __init__(self, corpora, tokenizer: BertTokenizer):

        self.corpora = corpora
        self.tokenizer = tokenizer

        d =  tokenizer.batch_encode_plus(corpora.sentences)
        
        padded_encoded_inputs = tokenizer.pad(d)
        
        self.input_ids = torch.tensor(padded_encoded_inputs.input_ids)
        self.attention_mask = torch.tensor(padded_encoded_inputs.attention_mask)
        self.labels = torch.tensor(corpora.labels)
        
    def __len__(self):
        return len(self.corpora.sentences)

    def __getitem__(self, idx):
        """
            Return the idx-th of the rows the self.input_ids, self.attention_mask, self.labels in this order.
            Don't do BERT tokenization here as that will be slow.
        """
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]

In [23]:
print(train_list[3000])

('5', 'i got this for my 3.5 year old nephew this christmas and he loved it. he immediately set off turning off all the lights in the room to see the stars. i got a glow in the dark constellation book to go with it- it was a total hit!')


In [24]:
counts = {1:0,2:0,3:0,4:0,5:0}
for i, j in test_list:
    counts[int(i)]+=1

counts

{1: 6350, 2: 3883, 3: 6695, 4: 16891, 5: 66181}

In [25]:
from torch import nn

class BERTClassifier(nn.Module):
    
    def __init__(self, BERT_model, hidden_layer_size, num_classes):
        
        super(BERTClassifier, self).__init__()

        # loaded pretrained model
        self.bert = BERT_model
        
        # simple neural network that convert embedding of the first token to a class
        self.classifier = nn.Sequential(
            nn.Linear(BERT_hidden_size, hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, num_classes)
        )
        
    def forward(self, input_ids, attention_mask):
        """
        The following two arguments are tensors from a mini-batch of the input_ids
            and attention_mask returned by the BERT tokenizer.
            
            input_ids: a tensor of shape [batch_size, max_length]
            attention_mask: a tensor of shape [batch_size, max_length]
            
            return: the logits of the sentences in the batch tensor of shape [batch_size, 1, 2]
        """
        # see https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        # and https://huggingface.co/docs/transformers/main_classes/output
        # for the return value of the forward function of BERT

        ### Your codes go here (30 points) ###
        z = self.bert.forward(input_ids, attention_mask)
        return torch.softmax(self.classifier(z[1]).unsqueeze(1), dim=-1) 
        
        return logits

In [26]:
# make sure you check the availability of GPU when you set the device ID.
device = torch.device(3)

BATCH_SIZE = 8
num_classes = 5
classifier_hidden_size = 128

## if using MAGIC
BERT_PATH = '/data/badassnlp/bert-base-uncased/'
## if using Google Colab, you need to load the bert model after it downloads the model files.
## 

BERT_hidden_size = 768

N_EPOCHS = 5
CLIP = 1

In [27]:
## 'uncased' means all words are lowered-cased before tokenization
## 'base' means the smaller version of BERT (12 layers, 16 heads)
## un-comment one of the following two options.

# if using MAGIC, load from local BERT folder
tokenizer = BertTokenizer.from_pretrained(BERT_PATH, local_files_only=True)
BERT_model = BertModel.from_pretrained(BERT_PATH, num_labels = 2, output_attentions = False, output_hidden_states = False
).to(device)

## if using Colab, load from automatically downloaded files. Downloading can take half a minute
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# BERT_model = BertModel.from_pretrained("bert-base-cased", num_labels = 2, output_attentions = False, output_hidden_states = False).to(device)


## if using MAGIC
print("creating training corpora ... ")
training_corpora = Corpora(train_list)
print("creating training dataset ... ")
training_dataset = TextClassificationDataSet(training_corpora, tokenizer)
print("creating training iterator ... ")
training_iterator = DataLoader(training_dataset, sampler = RandomSampler(training_dataset), batch_size=BATCH_SIZE)

## if using MAGIC
print("creating test corpora ... ")
dev_corpora = Corpora(test_list)
print("creating test dataset ... ")
dev_dataset = TextClassificationDataSet(dev_corpora, tokenizer)
print("creating test iterator ... ")
dev_iterator = DataLoader(dev_dataset, sampler = SequentialSampler(dev_dataset), batch_size=BATCH_SIZE)

classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)

Some weights of the model checkpoint at /data/badassnlp/bert-base-uncased/ were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


creating training corpora ... 


100%|██████████| 400000/400000 [00:00<00:00, 2297455.67it/s]


creating training dataset ... 
creating training iterator ... 
creating test corpora ... 


100%|██████████| 100000/100000 [00:00<00:00, 2182623.54it/s]

creating test dataset ... 


creating test iterator ... 


In [28]:
list(training_iterator)[0][0].shape

torch.Size([8, 174])

In [29]:
classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)
classifier.load_state_dict(torch.load("results/best_model_BERT.pt"))



<All keys matched successfully>

In [30]:
from torch import optim
criterion = nn.CrossEntropyLoss(reduction = 'sum')

from tqdm import tqdm

In [31]:
errors = []
confusion_matrix = []

def evaluate(model, iterator, criterion):

    model.eval()
    confusion_matrix.append(torch.zeros(5,5))
    
    for i, batch in tqdm(enumerate(iterator)):
        
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]-1).to(device)
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) 
        

        for i in range(len(y)):
            row = y[i]
            col = torch.argmax((logits.squeeze(1))[i])
            confusion_matrix[0][row][col] += 1
            if abs(row-col) >= 3:
                errors.append((ids[i], row, col))

            
 

In [32]:
print(list(dev_iterator)[0][0].shape)
print(list(dev_iterator)[0][1].shape)

torch.Size([8, 197])
torch.Size([8, 197])


In [33]:
evaluate(classifier, dev_iterator, criterion)   

12500it [09:00, 23.12it/s]


In [34]:
distance = 0
diagnal = 0

for i in range(5):
    for j in range(5):
        distance+= confusion_matrix[0][i][j]*abs(i-j)
        if i == j:
            diagnal += confusion_matrix[0][i][j]
        
print("dist:" ,distance)
print("diag: ",diagnal)

dist: tensor(150149.)
diag:  tensor(32968.)


In [35]:
confusion_matrix

[tensor([[ 2572.,  3200.,   193.,   148.,   237.],
         [  520.,  2728.,   341.,   133.,   161.],
         [  233.,  3639.,  1716.,   729.,   378.],
         [  188.,  7875.,  2110.,  4026.,  2692.],
         [  458., 34324.,  2578.,  6895., 21926.]])]

In [36]:
print(tokenizer.decode(errors[0][0]))
print("label: ", errors[0][1])
print("prediction", errors[0][2])

[CLS] very comfortable to sit in and is water repellled. wheels are sturdy and perfect for guest use seating as well as for the desk. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [37]:
print(tokenizer.decode(errors[10][0]))
print("label: ", errors[10][1])
print("prediction", errors[10][2])

[CLS] nice looking and not much on space needed. like the color and the toaster just toasted perfect. love this product it's worth the $ $ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [38]:
print(tokenizer.decode(errors[100][0]))
print("label: ", errors[100][1])
print("prediction", errors[100][2])

[CLS] i received the griddler as a gift for christmas with these waffle plates as an addition. what a nice surprise! used them immediately and think they are a great addition to the griddler. very easy to use and very easy to clean. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [39]:
print(tokenizer.decode(errors[1000][0]))
print("label: ", errors[1000][1])
print("prediction", errors[1000][2])

[CLS] " sturdy hard plastic, with easy to use latches, and a good solid handle. i will definitely feel a lot better about taking my sewing machine out of the house with this case to protect it. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [40]:
print(tokenizer.decode(errors[2000][0]))
print("label: ", errors[2000][1])
print("prediction", errors[2000][2])

[CLS] this is the third one i have owned in the last 7 or 8 years. i use it every day and it will keep coffee hot all day long. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [41]:
print(tokenizer.decode(errors[2001][0]))
print("label: ", errors[2001][1])
print("prediction", errors[2001][2])

[CLS] if you hate refilling your drink with ice everytime you refill you drink or don't like little ice cubes going in your mouth every time you take a sip this is for you [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [42]:
print(tokenizer.decode(errors[2005][0]))
print("label: ", errors[2005][1])
print("prediction", errors[2005][2])

[CLS] i was a bit skeptical of the satin sheets because they were so affordable and the thread count. i love these sateen sheets. now i can wear any kind of pj's without struggling to change position. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [43]:
print(tokenizer.decode(errors[2105][0]))
print("label: ", errors[2105][1])
print("prediction", errors[2105][2])

[CLS] " very easy to use instead of the normal fork, and the waffle comes right out at the first try without getting burned " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [44]:
print(tokenizer.decode(errors[1405][0]))
print("label: ", errors[1405][1])
print("prediction", errors[1405][2])

[CLS] i have had many different types of oven mitts over the years and these are the best i have ever had. they are thick and i haven't burned my hands yet ( a small miracle ). definitely would recommend them. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [45]:
print(tokenizer.decode(errors[1305][0]))
print("label: ", errors[1305][1])
print("prediction", errors[1305][2])

[CLS] " great little steamer. it was a breeze to set up and works just like it is supposed to. i read the reviews and have to admit i was a bit nervous, but knew of amazon's great return policies, so i got this steamer. i couldn't be happier. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [46]:
print(tokenizer.decode(errors[1306][0]))
print("label: ", errors[1306][1])
print("prediction", errors[1306][2])

[CLS] it dose make a difference when you use the hypo filter bags and not just the plain ones. and with 4 animals in and out my house you can tell. i will buy this again. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [47]:
print(tokenizer.decode(errors[1307][0]))
print("label: ", errors[1307][1])
print("prediction", errors[1307][2])

[CLS] " i found something similar with a cheaper price but the quality was also cheap. this one on the other hand, though a bit pricey, was of good quality. the strainer is not flexible which i like and the shine fits nicely with my new stainless steel single bowl sink. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [48]:
print(tokenizer.decode(errors[1308][0]))
print("label: ", errors[1308][1])
print("prediction", errors[1308][2])

[CLS] " you must make sure the washer does not stick, otherwise the pot will not perk. throughly wash stem after each use. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [49]:
print(tokenizer.decode(errors[1312][0]))
print("label: ", errors[1312][1])
print("prediction", errors[1312][2])

[CLS] " i do love the bag but the ice packs on mine are already ruined, they don't last very long they plastic is too hard and it breaks off causing leaks and the lids on the containers don't snap on so they just pop off. but the bag itself it great " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [50]:
print(tokenizer.decode(errors[312][0]))
print("label: ", errors[312][1])
print("prediction", errors[312][2])

[CLS] this is the best investment i have made towards my mixer since purchasing it! kitchenaid should send these with every mixer they sell - i wouldn't know what to do without this blade! thank you!! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [